<a href="https://colab.research.google.com/github/SampathViswanadha/IrisData/blob/master/mpi_multigranulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
#!pip3 install mpi4py
#!pip3 install bigmpi4py
%cd gdrive/My\ Drive/Colab\ Notebooks/

[Errno 2] No such file or directory: 'gdrive/My Drive/Colab Notebooks/'
/content


In [0]:
import bigmpi4py as BM
from mpi4py import MPI
import pandas as pd
from sklearn.utils import shuffle
import numpy as np
import random
import math


In [0]:
comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

In [0]:
global data_per_rank
global rows
global intersect_reduct
global other_reduct
global reduct
global broadcast_list

In [0]:
def decision_equivalence_classes(scattered_dataframe):
    dec_eq_classes = []
    col_names = scattered_dataframe.columns.values.tolist()
    dec_attr = col_names[-1]
    for dec_value in (pd.unique(scattered_dataframe[dec_attr])):
        dec_eq_classes.append(scattered_dataframe.index[scattered_dataframe[dec_attr] == dec_value].tolist())

    return dec_eq_classes

In [0]:
def indiscernibility(attr, table):

	u_ind = {}	#an empty dictionary to store the elements of the indiscernibility relation (U/IND({set of attributes}))
	attr_values = []	#an empty list to tore the values of the attributes

	for i in (table.index):

		attr_values = []
		for j in (attr):

			attr_values.append(table.loc[i, j])	#find the value of the table at the corresponding row and the desired attribute and add it to the attr_values list

		#convert the list to a string and check if it is already a key value in the dictionary
		key = ''.join(str(k) for k in (attr_values))

		if(key in u_ind):	#if the key already exists in the dictionary
			u_ind[key].add(i)

		else:	#if the key does not exist in the dictionary yet
			u_ind[key] = set()
			u_ind[key].add(i)

	return list(u_ind.values())



def lower_approximation(R, X):	#We have to try to describe the knowledge in X with respect to the knowledge in R; both are LISTS OS SETS [{},{}]

	l_approx = set()	#change to [] if you want the result to be a list of sets

	#print("X : " + str(len(X)))
	#print("R : " + str(len(R)))

	for i in range(len(X)):
		for j in range(len(R)):

			if(R[j].issubset(X[i])):
				l_approx.update(R[j])	#change to .append() if you want the result to be a list of sets

	return l_approx



def gamma_measure(describing_attributes, attributes_to_be_described, U, table):	#a functuon that takes attributes/features R, X, and the universe of objects

	f_ind = indiscernibility(describing_attributes, table)
	t_ind = indiscernibility(attributes_to_be_described, table)

	f_lapprox = lower_approximation(f_ind, t_ind)

	return len(f_lapprox)/len(U)

	#return mod_l_approx(l_approx)/len(U)


def quick_reduct(C, D, table, initial):	#C is the set of all conditional attributes; D is the set of decision attributes

	reduct = set(initial)

	gamma_C = gamma_measure(C, D, table.index, table)
	#print("The gamma measure of the Conditional attributes is :", gamma_C)
	gamma_R = 0

	while(gamma_R < gamma_C):

		T = reduct

		for x in (set(C) - reduct):

			feature = set()	#creating a new set to hold the currently selected feature
			feature.add(x)

			#print(feature)

			new_red = reduct.union(feature)	#directly unioning x separates the alphabets of the feature...

			gamma_new_red = gamma_measure(new_red, D, table.index, table)
			gamma_T = gamma_measure(T, D, table.index, table)

			if(gamma_new_red > gamma_T):

				T = reduct.union(feature)
				#print("added")

		reduct = T

		#finding the new gamma measure of the reduct

		gamma_R = gamma_measure(reduct, D, table.index, table)
		#print("The gamma of the reduct is:", gamma_R)

	return reduct

In [1]:
from google.colab import files
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving mpi_multi_granulation.py to mpi_multi_granulation (5).py
User uploaded file "mpi_multi_granulation.py" with length 8764 bytes


In [0]:
if __name__ == "__main__":

    global a 
    
    #if rank == 0:
    data = np.array([['case','Temp'     ,'Headache' ,'Weak' ,'Nausea' ,'Flu'],
                     [  1    ,'veryhigh'  ,'yes'     ,'yes'  ,'no'     ,'yes'],
                     [  2    ,'high'      ,'yes'     ,'no'   ,'yes'    ,'no'],
                     [  3    ,'normal'    ,'no'      ,'no'   ,'no'     ,'no'],
                     [  4    ,'normal'    ,'yes'     ,'yes'  ,'yes'    ,'yes'],
                     [  5    ,'high'      ,'no'      ,'yes'  ,'no'     ,'yes'],
                     [  6    ,'high'      ,'no'      ,'no'   ,'no'     ,'no'],
                     [  7    ,'normal'    ,'no'      ,'yes'  ,'no'     ,'yes'],
                     [  8    ,'veryhigh'  ,'no'      ,'yes'  ,'no'     ,'no']])
    '''a = pd.DataFrame(data=data[1:,0:],
                     index=data[1:,0],
                     columns=data[0,0:])'''
    
    #a = pd.read_csv("adult+stretch.data")
    a = pd.read_csv('agaricus-lepiota.csv')
    #a = pd.read_csv('connect-4.data')
    #a = pd.read_csv('balance-scale.csv')
    #a = pd.read_csv('led-creator.data')
    #a = pd.read_csv('breast-cancer.csv')

    cols = a.columns.tolist()
    '''if cols[0] == 'dec_variable':
        a = a.drop(columns = ['dec_variable']).assign(dec_variable = a['dec_variable'])'''
    #a = a.drop(columns = ['dec_variable']).assign(dec_variable = a['dec_variable'])
    '''a = pd.read_csv("connect-4.data")
    a['index'] = a.index
    cols = a.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    a = a[cols]'''


In [17]:
shuffled_df = shuffle(a)

if rank == 0:
  print('Shuffled dataframe: \n%s' % (shuffled_df))

Shuffled dataframe: 
     cap-shape cap-surface cap-color  ... population habitat dec_variable
6159         x           s         e  ...          v       d            p
178          x           y         y  ...          n       m            e
7086         b           f         g  ...          s       g            e
7879         k           s         n  ...          v       d            p
402          x           y         n  ...          v       u            p
...        ...         ...       ...  ...        ...     ...          ...
294          x           s         w  ...          s       m            e
1586         x           f         n  ...          v       d            e
4148         f           y         g  ...          v       d            p
1947         x           y         g  ...          y       d            e
2847         x           f         n  ...          y       d            e

[8124 rows x 23 columns]


In [18]:
    dec_eq_classes = decision_equivalence_classes(shuffled_df)

    dummy = []
    original_reduct = quick_reduct(a.columns[1:-1], [a.columns[-1]], a, dummy)
        

    if rank == 0:
      print('The original reduct is: ', original_reduct)

    reduct = []
    broadcast_list = []
    rows = shuffled_df.index.values.tolist()

The original reduct is:  {'spore-print-color', 'stalk-surface-above-ring', 'odor', 'gill-size', 'stalk-color-below-ring'}


In [0]:
def sliceit(lis, chunk_num):
  start = 0
  for i in range(chunk_num):
    stop = start + len(lis[i::chunk_num])
    yield lis[start:stop]
    start = stop
  
chunk = size + 1
sam = sliceit(rows, chunk)
sam_list = []
for x in sam:
  sam_list.append(x)

for l in range(len(sam_list) - 1):
  sam_list[l].extend(sam_list[-1])

sam_list.remove(sam_list[-1])

In [20]:
scatter_list = BM.scatter(sam_list, comm)
print('Rank %s: scattered list \n%s\n' % (rank, scatter_list))

df_for_each = shuffled_df.loc[scatter_list[0], :]

Rank 0: scattered list 
[[6159, 178, 7086, 7879, 402, 7026, 6470, 3583, 5433, 2094, 6031, 596, 4510, 4605, 830, 6910, 5309, 6451, 333, 4028, 5330, 1108, 799, 3550, 2639, 5948, 1696, 6989, 4322, 3311, 4505, 2709, 473, 6030, 567, 5235, 3388, 1459, 7269, 7388, 5873, 3127, 59, 4008, 6491, 4919, 1503, 7383, 6465, 5263, 270, 4497, 241, 7430, 1972, 4010, 792, 4770, 5621, 2667, 3500, 4021, 1277, 4607, 6428, 2819, 1855, 1186, 3579, 7607, 5699, 3609, 3042, 2474, 5960, 4552, 3885, 8013, 5012, 3835, 392, 5414, 7167, 2257, 6787, 6419, 4726, 4430, 7688, 5509, 7328, 6014, 7144, 1523, 5081, 7482, 6570, 3636, 5595, 5034, 4327, 4571, 170, 2186, 3088, 1871, 1075, 4212, 2990, 7478, 4656, 7093, 2336, 3248, 6008, 7081, 1509, 6059, 4979, 3357, 3741, 7238, 6619, 1247, 456, 4124, 114, 1030, 5951, 1388, 3991, 6796, 2075, 180, 3923, 3984, 8114, 276, 757, 4554, 522, 4358, 6347, 1181, 3425, 2160, 2136, 2009, 3886, 3351, 4365, 7581, 5503, 2728, 5026, 3705, 329, 3799, 2598, 1937, 7380, 518, 3744, 4813, 5546, 3280, 7

In [21]:
if rank == 0:
        
  reduct = quick_reduct(df_for_each.columns[1:-1], [df_for_each.columns[-1]], df_for_each, broadcast_list)
  #print('The reduct obtained for rank %d is %s' % (rank, reduct))
  for i in range(1, size):
    other_reduct = comm.recv(source = i)
    reduct = reduct.intersection(other_reduct)
  print('The reduct after intersection of all the reducts obtained from all processors is %s:' % (reduct))
        
else:
  inter = quick_reduct(df_for_each.columns[1:-1], [df_for_each.columns[-1]], df_for_each, broadcast_list)
  #print('The reduct obtained for rank %d is %s' % (rank, inter))
  comm.send(inter, dest = 0)

broadcast_list = comm.bcast(reduct, root = 0)


The reduct after intersection of all the reducts obtained from all processors is {'spore-print-color', 'stalk-surface-above-ring', 'odor', 'gill-size', 'stalk-color-below-ring'}:


In [22]:
if rank == 0:
        
  reduct = quick_reduct(df_for_each.columns[1:-1], [df_for_each.columns[-1]], df_for_each, broadcast_list)
  print('The reduct obtained for rank %d is %s' % (rank, reduct))
  for i in range(1, size):
    other_reduct = comm.recv(source = i)
    reduct = reduct.union(other_reduct)
  print('The reduct after fusion of all the reducts obtained from all processors is %s:' % (reduct))
        
else:
  inter = quick_reduct(df_for_each.columns[1:-1], [df_for_each.columns[-1]], df_for_each, broadcast_list)
  print('The reduct obtained for rank %d is %s' % (rank, inter))
  comm.send(inter, dest = 0)


The reduct obtained for rank 0 is {'spore-print-color', 'stalk-surface-above-ring', 'odor', 'gill-size', 'stalk-color-below-ring'}
The reduct after fusion of all the reducts obtained from all processors is {'spore-print-color', 'stalk-surface-above-ring', 'odor', 'gill-size', 'stalk-color-below-ring'}:


In [3]:
!mpirun --allow-run-as-root -np 4 python3 mpi_multi_granulation.py 

Traceback (most recent call last):
  File "mpi_multi_granulation.py", line 155, in <module>
    a = pd.read_csv('agaricus-lepiota.data')
  File "/usr/local/lib/python3.6/dist-packages/pandas/io/parsers.py", line 676, in parser_f
    return _read(filepath_or_buffer, kwds)
  File "/usr/local/lib/python3.6/dist-packages/pandas/io/parsers.py", line 448, in _read
    parser = TextFileReader(fp_or_buf, **kwds)
  File "/usr/local/lib/python3.6/dist-packages/pandas/io/parsers.py", line 880, in __init__
    self._make_engine(self.engine)
  File "/usr/local/lib/python3.6/dist-packages/pandas/io/parsers.py", line 1114, in _make_engine
    self._engine = CParserWrapper(self.f, **self.options)
  File "/usr/local/lib/python3.6/dist-packages/pandas/io/parsers.py", line 1891, in __init__
    self._reader = parsers.TextReader(src, **kwds)
  File "pandas/_libs/parsers.pyx", line 374, in pandas._libs.parsers.TextReader.__cinit__
  File "pandas/_libs/parsers.pyx", line 674, in pandas._libs.parsers.TextRea